<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Using-bigger-data" data-toc-modified-id="Using-bigger-data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Using bigger data</a></span><ul class="toc-item"><li><span><a href="#Training-data" data-toc-modified-id="Training-data-0.1.1"><span class="toc-item-num">0.1.1&nbsp;&nbsp;</span>Training data</a></span></li></ul></li></ul></li><li><span><a href="#Using-bktree" data-toc-modified-id="Using-bktree-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using bktree</a></span></li><li><span><a href="#No-bktree" data-toc-modified-id="No-bktree-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>No bktree</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
ls ..

__init__.py   data/         notebooks/    pyngramspell/


In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [4]:
import data
from data import utils

train, test = utils.get_evaluation()

X_tr = [x['original'] for x in train]
y_tr = [x['corrected'] for x in train]
X_te = [x['original'] for x in test]
y_te = [x['corrected'] for x in test]

train_data = [' '.join(x) for x in y_tr]

In [5]:
import pyngramspell
speller = pyngramspell.PyNgramSpell()

In [6]:
speller.fit(train_data)

In [7]:
speller.transform('the maan went to the store')

['the', 'man', 'went', 'to', 'the', 'store']

In [8]:
speller.transform('red shirtt')

['red', 'shirtt']

In [9]:
%%time 
speller.transform('red shirtt')

CPU times: user 194 µs, sys: 25 µs, total: 219 µs
Wall time: 236 µs


['red', 'shirtt']

### Using bigger data

#### Training data


The dataset used to train the model comes from  https://jmcauley.ucsd.edu/data/amazon/

The `Video_Games_5.json` can be downloaded from 
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz

In [10]:
import pandas as pd
from os.path import expanduser
home = expanduser("~")

In [11]:
dataset_path = os.path.join(home,'Datasets/amazon_product_data/reviews_Video_Games_5.json')
df = pd.read_json(dataset_path, lines=True)

In [12]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4,DIRT 3,1308009600,"06 14, 2011"


In [13]:
train_data = list(df['reviewText'])

## Using bktree

In [14]:
import pyngramspell
speller_bktree = pyngramspell.PyNgramSpell(min_freq=20,
                                           bktree=True)

speller_bktree.fit(train_data)

In [15]:
len(speller_bktree.vocabulary)

26484

In [16]:
speller_bktree.transform('xboj 360 colector edition')

['xbox', '360', 'collector', 'edition']

In [17]:
speller_bktree.transform('pssp blac edition')

['psp', 'black', 'edition']

In [18]:
speller_bktree.transform('new playstation game batmen')

['new', 'playstation', 'game', 'batman']

In [19]:
 speller_bktree.get_candidates("pleystation", max_dist=3)

['playstion',
 'levitation',
 'playstaion',
 'playstation',
 'playstation3',
 'playstation2',
 'playstations']

In [20]:
%%timeit
speller_bktree.get_candidates("pleystation", max_dist=3)

13.8 ms ± 80.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
ind_longest = np.argmax([x for x in map(len,list(speller_bktree.vocabulary))])

In [37]:
list(speller_bktree.vocabulary)[ind_longest]

'multiplayeroverall'

In [38]:
%%timeit
speller_bktree.get_candidates("multyplayeroveral", max_dist=3)

225 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## No bktree

In [22]:
speller_nobktree = pyngramspell.PyNgramSpell(min_freq=20,
                                           bktree=False)

speller_nobktree.fit(train_data)

In [23]:
speller_nobktree.transform('xboj 360 colector edition')

['xbox', '360', 'collector', 'edition']

In [24]:
speller_nobktree.transform('pssp blac edition')

['psp', 'black', 'edition']

In [25]:
speller_nobktree.transform('new playstation game batmen')

['new', 'playstation', 'game', 'batman']

In [26]:
 speller_nobktree.get_candidates("pleystation", max_dist=3)

['playstion',
 'playstaion',
 'playstation3',
 'levitation',
 'playstation',
 'playstation2',
 'playstations']

In [27]:
%%timeit
speller_nobktree.get_candidates("pleystation", max_dist=3)

56.5 ms ± 445 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%%timeit
speller_nobktree.get_candidates("multyplayeroveral", max_dist=3)

60.5 ms ± 659 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
